In [2]:
!pip install scikit-learn

You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [6]:
%xmode plain
try:
    1/0
except:
    compile(".a", '', 'exec')

Exception reporting mode: Plain


SyntaxError: invalid syntax (<string>, line 1)

In [7]:
import tensorflow as tf

SyntaxError: invalid syntax (python_message.py, line 846)

In [4]:
import scipy.io
import numpy as np
import os
import random
import math
#from skimage.measure import structural_similarity as ssim
#from sporco import util
#import sklearn
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import linear_model
from sklearn.feature_selection import RFECV, RFE
import multiprocessing
import datetime
import hdf5storage

#import xgboost as xgb

In [2]:
#Random Forest

def rfc(train_data, train_label, test_data, test_label):
    rf = RandomForestClassifier(n_estimators=150,
                                    criterion='gini',
                                    max_depth=None,
                                    min_samples_split=2,
                                    min_samples_leaf=1,
                                    min_weight_fraction_leaf=0.0,
                                    max_features=None,
                                    max_leaf_nodes=None,
                                    bootstrap=True,
                                    oob_score=False,
                                    n_jobs=10,
                                    random_state=123,
                                    verbose=0,
                                    warm_start=False,
                                    class_weight=None)
    rf.fit(train_data, train_label.ravel())
    result = rf.predict_proba(test_data)
    acc = 0.0
    for i in range(np.shape(test_data)[0]):
        r = np.argmax(result[i])
        if r == test_label[i]:
            acc += 1
    acc /= np.shape(test_data)[0]
    acc *= 100
    return acc, result

In [13]:
flag = 0
print("Initialize data")
for test_index in t_index:
    if flag == Outer_loop-1:
        test_index = np.array(np.concatenate((test_index,
                                              np.array(range(int(np.shape(data)[0]/Outer_loop)*Outer_loop,
                                                                         np.shape(data)[0]))), axis=0), dtype=np.int32)
        
    train_index = np.setdiff1d(np.array(range(0,np.shape(data)[0])), test_index)
    train_data = data[train_index]
    train_label = label[train_index]
    test_data = data[test_index]
    test_label = label[test_index]

Initialize data


In [16]:
rf_acc, result_rf = rfc(train_data, train_label, test_data, test_label)

In [41]:
np.shape(test_data)[0]

700

In [18]:
rf_acc

1.0

In [42]:
result = rf.predict_proba(test_data)

NameError: name 'rf' is not defined

In [37]:
result_rf[0]

array([0.01333333, 0.22666667, 0.00666667, 0.00666667, 0.        ,
       0.02      , 0.02666667, 0.        , 0.09333333, 0.01333333,
       0.        , 0.12666667, 0.        , 0.00666667, 0.05333333,
       0.00666667, 0.01333333, 0.04      , 0.03333333, 0.06666667,
       0.02666667, 0.07333333, 0.        , 0.00666667, 0.01333333,
       0.00666667, 0.        , 0.1       , 0.        , 0.02      ,
       0.        ])

In [35]:
np.argmax(result_rf[0][0])

0

In [25]:
len(test_label)

700

In [40]:
a = np.array([[2,5,6],[7,6,1]])
print(np.argmax(a))

3


In [33]:
test_label

array([[ 2],
       [ 8],
       [12],
       [24],
       [23],
       [ 1],
       [23],
       [ 2],
       [ 6],
       [13],
       [15],
       [ 7],
       [ 8],
       [13],
       [12],
       [ 2],
       [ 4],
       [ 8],
       [18],
       [ 3],
       [ 3],
       [17],
       [ 4],
       [23],
       [12],
       [12],
       [17],
       [ 5],
       [25],
       [23],
       [21],
       [22],
       [12],
       [25],
       [ 4],
       [12],
       [ 5],
       [30],
       [ 5],
       [ 9],
       [ 3],
       [12],
       [31],
       [23],
       [14],
       [ 2],
       [27],
       [12],
       [12],
       [13],
       [16],
       [ 5],
       [12],
       [ 5],
       [ 2],
       [14],
       [13],
       [24],
       [12],
       [12],
       [23],
       [ 3],
       [18],
       [ 3],
       [12],
       [12],
       [25],
       [19],
       [17],
       [ 6],
       [12],
       [18],
       [20],
       [23],
       [12],
       [21],
       [17],

In [6]:
#NN MODEL

def fnn(x, input_size, output_size, keep_prob, stddev=0.01, constant=0.0001, dropout=True, end=False):
    fc_w = tf.Variable(tf.truncated_normal([input_size,output_size], stddev=stddev,seed=np.random.seed(2018)))
    fc_b = tf.Variable(tf.constant(constant,shape=[output_size]), dtype=tf.float32)
    fc_h = tf.nn.relu(tf.matmul(x,fc_w)+fc_b) if not end else tf.matmul(x,fc_w)+fc_b
    return tf.nn.dropout(fc_h, keep_prob,seed=np.random.seed(2018)) if dropout else fc_h

def fcn(x, input_size, output_size, nlayers, nparameters, keep_prob):
    if nlayers == 1:
        h1 = fnn(x, input_size, output_size, keep_prob, end=True)
    elif nlayers == 2:
        h1 = fnn(fnn(x, input_size, nparameters, keep_prob, end=False), nparameters, output_size, keep_prob, end=True)
    elif nlayers >= 3:
        h0 = fnn(x, input_size, nparameters, keep_prob, end=False)
        for j in range(0,nlayers-2):
            if j == 0:
                h1 = fnn(h0, nparameters, nparameters, keep_prob, end=False)
            else:
                h1 = fnn(h1, nparameters, nparameters, keep_prob, end=False)
        h1 = fnn(h1, nparameters, output_size, keep_prob, end=True)
    else:
        print("# of layers can't be smaller than 0")
    return h1

def dnn(train_data, train_label, test_data, test_label):
    g = tf.Graph()
    config = tf.ConfigProto()
    config.gpu_options.allow_growth=True
    batch_size = 10
    input_size = np.shape(train_data)[1]
    output_size = 31

    with g.as_default():
        p_x = tf.placeholder(tf.float32, [batch_size, 1, input_size, 1])
        p_y = tf.placeholder(tf.float32, [batch_size, output_size])
        keep_prob = tf.placeholder(tf.float32)
        h10_flat = tf.reshape(p_x, [batch_size,-1])
        h1 = fnn(h10_flat, input_size, 2048, keep_prob, end=False)
        h2 = fnn(h1, 2048, 2048, keep_prob, end=False)
        h3 = fnn(h2, 2048, 31, keep_prob, end=True)
        h4 = tf.reshape(h3, [batch_size, 31])
        h_c = tf.nn.softmax(h4)
        loss = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(labels=p_y, logits=h4))
        optim = tf.train.AdamOptimizer(1e-5)
        trainer = optim.minimize(loss)
    
    accuracy = 0.0
    #600*31
    result = np.zeros([np.shape(test_data)[0], 31])
    with tf.Session(graph=g, config=config) as sess:
        sess.run(tf.global_variables_initializer())
        #120代
        for e in range(0,120):
            loss_tot = 0.0
            for i in range(0,int(np.ceil(np.shape(train_data)[0]/batch_size))):
                a = np.random.randint(0,np.shape(train_data)[0],size=batch_size)
                x = train_data[a].reshape([batch_size, 1, input_size, 1])#[4,1,18181,1]
                y = np.zeros([batch_size, output_size])
                index = train_label[a]
                for u in range(0,batch_size):
                    y[u,index[u]] = 1
                _ , loss_val = sess.run([trainer, loss], feed_dict={p_x:x, p_y:y, keep_prob:0.6})
                loss_tot += loss_val
            print("%d epoch Loss: %f" % (e,(loss_tot)/np.shape(train_data)[0]))
        temp = 0
        for i in range(0,int(np.floor(np.shape(test_data)[0]/batch_size))):
            x = test_data[i*batch_size:(i+1)*batch_size].reshape([batch_size, 1, input_size, 1])
            out = sess.run(h_c, feed_dict={p_x:x, keep_prob:1})
            for j in range(0, batch_size):
                t = np.squeeze(out[j])
                result[temp] = t
                temp+=1
        remain = int(np.shape(test_data)[0]-np.floor(np.shape(test_data)[0]/batch_size)*batch_size)
        if remain > 0:
            x = test_data[-batch_size-1:-1].reshape([batch_size, 1, input_size, 1])
            out = sess.run(h_c, feed_dict={p_x:x, keep_prob:1})
            for j in range(0,int(remain)):
                t = np.squeeze(out[j+(batch_size-remain)])
                result[temp] = t
                temp+=1
        for i in range(0,np.shape(test_data)[0]):
            ind = np.argmax(np.squeeze(result[i]))
            if ind == test_label[i]:
                accuracy += 1
        accuracy /= np.shape(test_data)[0]*0.01
        sess.close()
    return accuracy, result

In [6]:
# def lrelu(x, leak=0.2):
#     f1 = 0.5*(1+leak)
#     f2 = 0.5*(1-leak)
#     return f1*x+f2*abs(x)





In [4]:
dataID = hdf5storage.loadmat('data.mat')
data = np.array(dataID['data'], dtype=np.float32)
gt1 = scipy.io.loadmat('label.mat')
label = np.array(gt1['label'], dtype=np.int32)

In [5]:
Outer_loop = 10
Inner_loop = 10

In [6]:
t_index = np.random.permutation(int(np.shape(data)[0]/Outer_loop)*Outer_loop)
t_index = np.reshape(t_index, [Outer_loop, -1])

In [7]:
np.zeros([Inner_loop,5], dtype=np.float32)

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]], dtype=float32)

In [8]:
tot_acc = np.zeros([Inner_loop,5], dtype=np.float32)
u = np.sum(tot_acc,0)

In [10]:
len(u)

5

In [ ]:
box = np.array([500, 1000, 4000, 8000, 35565], dtype=np.int32)
flag = 0
print("Initialize data")
for test_index in t_index:
    if flag == Outer_loop-1:
        test_index = np.array(np.concatenate((test_index,
                                              np.array(range(int(np.shape(data)[0]/Outer_loop)*Outer_loop,
                                                                         np.shape(data)[0]))), axis=0), dtype=np.int32)
        
    train_index = np.setdiff1d(np.array(range(0,np.shape(data)[0])), test_index)
    train_data = data[train_index]
    train_label = label[train_index]
    test_data = data[test_index]
    test_label = label[test_index]

    # shape 重排
    kf = np.random.permutation(int(np.shape(train_data)[0]/Inner_loop)*Inner_loop)
    # [10,-1] 6300 重排 row:10 column:自己除 
    kf = kf.reshape([Inner_loop]+[-1])
    # 6302*48
    # val_result = np.zeros([np.shape(train_data)[0],48], dtype=np.float32)

    #10*5 [[0,0,...]...]
    tot_acc = np.zeros([Inner_loop,5], dtype=np.float32)
    lsvc = LinearSVC(C=1, penalty="l1", dual=False).fit(data, label)
    coef = np.squeeze(np.sum(np.square(np.array(lsvc.coef_)), axis=0))
    coefidx = np.argsort(coef)
    for inner_fold in range(0,Inner_loop):
        val_test_ind = kf[inner_fold]
        if inner_fold == Inner_loop-1:
            val_test_ind = np.array(np.concatenate((val_test_ind,np.array(range(int(np.shape(train_data)[0]/Outer_loop)*Outer_loop,np.shape(train_data)[0]),dtype=np.int32)), axis=0),dtype=np.int32)

        val_train_ind = np.setdiff1d(np.array(range(0,np.shape(train_data)[0]),dtype=np.int32), val_test_ind)
        val_train = train_data[val_train_ind]
        val_test = train_data[val_test_ind]
        val_train_label = train_label[val_train_ind]
        val_test_label = train_label[val_test_ind]
        temp = 0
        for item in box:
            idx = coefidx[-item:]
            vtrain = val_train[:,idx]
            vtest = val_test[:,idx]
            nn_acc, result_nn = dnn(vtrain, val_train_label, vtest, val_test_label)
            rf_acc, result_rf = rfc(vtrain, val_train_label, vtest, val_test_label)
            en_acc = 0.0
            for i in range(0,np.shape(vtest)[0]):
                r = np.argmax(result_nn[i]+result_rf[i])
                if r == val_test_label[i]:
                    en_acc += 1
            en_acc /= np.shape(vtest)[0]*0.01
            tot_acc[inner_fold,temp] = en_acc
            print("Inner_fold # of features: %d, Neural network accuracy: %f, Random forests accuracy: %f, Ensemble accuracy: %f" % (item, nn_acc, rf_acc, en_acc))
            temp += 1

    u = np.sum(tot_acc,0)

    best_n = box[np.argmax(u)]
    idx = coefidx[-best_n:]

    tr_data = train_data[:,idx]
    te_data = test_data[:,idx]
    nn_acc, result_nn = dnn(tr_data, train_label, te_data, test_label)
    rf_acc, result_rf = rfc(tr_data, train_label, te_data, test_label)
    en_acc = 0.0
    for i in range(0,np.shape(te_data)[0]):
        r = np.argmax(result_nn[i]+result_rf[i])
        if r == test_label[i]:
            en_acc += 1
    en_acc /= np.shape(te_data)[0]*0.01
    print("Outer_fold # of features:  %d, Neural network accuracy: %f, Random forests accuracy: %f, Ensemble accuracy: %f" % (best_n, nn_acc, rf_acc, en_acc))
    flag += 1

In [7]:

if __name__ == "__main__":

    
#     dataID = hdf5storage.loadmat('data.mat')
#     data = np.array(dataID['data'], dtype=np.float32)
#     gt1 = scipy.io.loadmat('label.mat')
#     label = np.array(gt1['label'], dtype=np.int32)
    
    
    #Initialize
    label -= 1
    np.random.seed(2018)

    Outer_loop = 10
    Inner_loop = 10
    
    t_index = np.random.permutation(int(np.shape(data)[0]/Outer_loop)*Outer_loop)
    t_index = np.reshape(t_index, [Outer_loop, -1])
    os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
    os.environ["CUDA_VISIBLE_DEVICES"]="0" 
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
    config = tf.ConfigProto()
    config.gpu_options.allow_growth=True
    
    box = np.array([500, 1000, 4000, 8000, 35565], dtype=np.int32)
    flag = 0
    print("Initialize data")
    for test_index in t_index:
        
        if flag == Outer_loop-1:
            print(flag)
            test_index = np.array(np.concatenate((test_index, np.array(range(int(np.shape(data)[0]/Outer_loop)*Outer_loop,np.shape(data)[0]))), axis=0), dtype=np.int32)
        train_index = np.setdiff1d(np.array(range(0,np.shape(data)[0])), test_index)
        train_data = data[train_index]
        train_label = label[train_index]
        test_data = data[test_index]
        test_label = label[test_index]
        
        kf = np.random.permutation(int(np.shape(train_data)[0]/Inner_loop)*Inner_loop)
        kf = kf.reshape([Inner_loop]+[-1])
        val_result = np.zeros([np.shape(train_data)[0],48], dtype=np.float32)
        
        tot_acc = np.zeros([Inner_loop,5], dtype=np.float32)
        print("LSVC")
        lsvc = LinearSVC(C=1, penalty="l1", dual=False).fit(data, label)
        print("LSVC_end")
        coef = np.squeeze(np.sum(np.square(np.array(lsvc.coef_)), axis=0))
        coefidx = np.argsort(coef)
#         for inner_fold in range(0,Inner_loop):
#             val_test_ind = kf[inner_fold]
#             if inner_fold == Inner_loop-1:
#                 val_test_ind = np.array(np.concatenate((val_test_ind,np.array(range(int(np.shape(train_data)[0]/Outer_loop)*Outer_loop,np.shape(train_data)[0]),dtype=np.int32)), axis=0),dtype=np.int32)
            
#             val_train_ind = np.setdiff1d(np.array(range(0,np.shape(train_data)[0]),dtype=np.int32), val_test_ind)
#             val_train = train_data[val_train_ind]
#             val_test = train_data[val_test_ind]
#             val_train_label = train_label[val_train_ind]
#             val_test_label = train_label[val_test_ind]
#             temp = 0
#             for item in box:
#                 idx = coefidx[-item:]
#                 vtrain = val_train[:,idx]
#                 vtest = val_test[:,idx]
#                 nn_acc, result_nn = dnn(vtrain, val_train_label, vtest, val_test_label)
#                 rf_acc, result_rf = rfc(vtrain, val_train_label, vtest, val_test_label)
#                 en_acc = 0.0
#                 for i in range(0,np.shape(vtest)[0]):
#                     r = np.argmax(result_nn[i]+result_rf[i])
#                     if r == val_test_label[i]:
#                         en_acc += 1
#                 en_acc /= np.shape(vtest)[0]*0.01
#                 tot_acc[inner_fold,temp] = en_acc
#                 print("Inner_fold # of features: %d, Neural network accuracy: %f, Random forests accuracy: %f, Ensemble accuracy: %f" % (item, nn_acc, rf_acc, en_acc))
#                 temp += 1
        
        u = np.sum(tot_acc,0)
       
        best_n = box[np.argmax(u)]
        idx = coefidx[-best_n:]
        
        tr_data = train_data[:,idx]
        te_data = test_data[:,idx]
        print("NN")
        nn_acc, result_nn = dnn(tr_data, train_label, te_data, test_label)
        print("rf")
        rf_acc, result_rf = rfc(tr_data, train_label, te_data, test_label)
        en_acc = 0.0
        for i in range(0,np.shape(te_data)[0]):
            r = np.argmax(result_nn[i]+result_rf[i])
            if r == test_label[i]:
                en_acc += 1
        en_acc /= np.shape(te_data)[0]*0.01
        print("Outer_fold # of features:  %d, Neural network accuracy: %f, Random forests accuracy: %f, Ensemble accuracy: %f" % (best_n, nn_acc, rf_acc, en_acc))
        flag += 1

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 3319, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-d2575f9a884d>", line 11, in <module>
    label -= 1
NameError: name 'label' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2034, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/ultratb.py", line 1151, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/ultratb.py", line 319, i

NameError: name 'label' is not defined

In [ ]:
if __name__ == "__main__":
#Load data
    print("Load data")
    dataID = hdf5storage.loadmat('data.mat')
    data = np.array(dataID['data'], dtype=np.float32)
    gt1 = scipy.io.loadmat('label.mat')
    label = np.array(gt1['label'], dtype=np.int32)
    print("Load data")
    
    #Initialize
    label -= 1
    np.random.seed(2018)

    Outer_loop = 10
    Inner_loop = 10
    
    t_index = np.random.permutation(int(np.shape(data)[0]/Outer_loop)*Outer_loop)
    t_index = np.reshape(t_index, [Outer_loop, -1])
    os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
    os.environ["CUDA_VISIBLE_DEVICES"]="0" 
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
    config = tf.ConfigProto()
    config.gpu_options.allow_growth=True
    
    box = np.array([500, 1000, 4000, 8000, 35565], dtype=np.int32)
    flag = 0
    print("Initialize data")

In [16]:

if __name__ == "__main__":
#Load data
    print("Load data")
    dataID = hdf5storage.loadmat('data.mat')
    data = np.array(dataID['data'], dtype=np.float32)
    gt1 = scipy.io.loadmat('label.mat')
    label = np.array(gt1['label'], dtype=np.int32)
    print("Load data")
    
    #Initialize
    label -= 1
    np.random.seed(2018)

    Outer_loop = 10
    Inner_loop = 10
    
    t_index = np.random.permutation(int(np.shape(data)[0]/Outer_loop)*Outer_loop)
    t_index = np.reshape(t_index, [Outer_loop, -1])
    os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
    os.environ["CUDA_VISIBLE_DEVICES"]="0" 
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
    config = tf.ConfigProto()
    config.gpu_options.allow_growth=True
    
    box = np.array([500, 1000, 4000, 8000, 35565], dtype=np.int32)
    flag = 0
    print("Initialize data")
#     for test_index in t_index:
#         if flag == Outer_loop-1:
#             test_index = np.array(np.concatenate((test_index, np.array(range(int(np.shape(data)[0]/Outer_loop)*Outer_loop,np.shape(data)[0]))), axis=0), dtype=np.int32)
#         train_index = np.setdiff1d(np.array(range(0,np.shape(data)[0])), test_index)
#         train_data = data[train_index]
#         train_label = label[train_index]
#         test_data = data[test_index]
#         test_label = label[test_index]
        
#         kf = np.random.permutation(int(np.shape(train_data)[0]/Inner_loop)*Inner_loop)
#         kf = kf.reshape([Inner_loop]+[-1])
#         val_result = np.zeros([np.shape(train_data)[0],48], dtype=np.float32)
        
#         tot_acc = np.zeros([Inner_loop,5], dtype=np.float32)
#         lsvc = LinearSVC(C=1, penalty="l1", dual=False).fit(data, label)
#         coef = np.squeeze(np.sum(np.square(np.array(lsvc.coef_)), axis=0))
#         coefidx = np.argsort(coef)
#         for inner_fold in range(0,Inner_loop):
#             val_test_ind = kf[inner_fold]
#             if inner_fold == Inner_loop-1:
#                 val_test_ind = np.array(np.concatenate((val_test_ind,np.array(range(int(np.shape(train_data)[0]/Outer_loop)*Outer_loop,np.shape(train_data)[0]),dtype=np.int32)), axis=0),dtype=np.int32)
            
#             val_train_ind = np.setdiff1d(np.array(range(0,np.shape(train_data)[0]),dtype=np.int32), val_test_ind)
#             val_train = train_data[val_train_ind]
#             val_test = train_data[val_test_ind]
#             val_train_label = train_label[val_train_ind]
#             val_test_label = train_label[val_test_ind]
#             temp = 0
#             for item in box:
#                 idx = coefidx[-item:]
#                 vtrain = val_train[:,idx]
#                 vtest = val_test[:,idx]
#                 nn_acc, result_nn = dnn(vtrain, val_train_label, vtest, val_test_label)
#                 rf_acc, result_rf = rfc(vtrain, val_train_label, vtest, val_test_label)
#                 en_acc = 0.0
#                 for i in range(0,np.shape(vtest)[0]):
#                     r = np.argmax(result_nn[i]+result_rf[i])
#                     if r == val_test_label[i]:
#                         en_acc += 1
#                 en_acc /= np.shape(vtest)[0]*0.01
#                 tot_acc[inner_fold,temp] = en_acc
#                 print("Inner_fold # of features: %d, Neural network accuracy: %f, Random forests accuracy: %f, Ensemble accuracy: %f" % (item, nn_acc, rf_acc, en_acc))
#                 temp += 1
        
#         u = np.sum(tot_acc,0)
       
#         best_n = box[np.argmax(u)]
#         idx = coefidx[-best_n:]
        
#         tr_data = train_data[:,idx]
#         te_data = test_data[:,idx]
#         nn_acc, result_nn = dnn(tr_data, train_label, te_data, test_label)
#         rf_acc, result_rf = rfc(tr_data, train_label, te_data, test_label)
#         en_acc = 0.0
#         for i in range(0,np.shape(te_data)[0]):
#             r = np.argmax(result_nn[i]+result_rf[i])
#             if r == test_label[i]:
#                 en_acc += 1
#         en_acc /= np.shape(te_data)[0]*0.01
#         print("Outer_fold # of features:  %d, Neural network accuracy: %f, Random forests accuracy: %f, Ensemble accuracy: %f" % (best_n, nn_acc, rf_acc, en_acc))
#         flag += 1

Load data
Load data
Initialize data


In [10]:
    for test_index in t_index:
        if flag == Outer_loop-1:
            test_index = np.array(np.concatenate((test_index, np.array(range(int(np.shape(data)[0]/Outer_loop)*Outer_loop,np.shape(data)[0]))), axis=0), dtype=np.int32)
        train_index = np.setdiff1d(np.array(range(0,np.shape(data)[0])), test_index)
        train_data = data[train_index]
        train_label = label[train_index]
        test_data = data[test_index]
        test_label = label[test_index]
        print("KF")
        kf = np.random.permutation(int(np.shape(train_data)[0]/Inner_loop)*Inner_loop)
        kf = kf.reshape([Inner_loop]+[-1])
        val_result = np.zeros([np.shape(train_data)[0],48], dtype=np.float32)
        print("tot_acc")
        tot_acc = np.zeros([Inner_loop,5], dtype=np.float32)
        print("LSVC")
        lsvc = LinearSVC(C=1, penalty="l1", dual=False).fit(data, label)
        coef = np.squeeze(np.sum(np.square(np.array(lsvc.coef_)), axis=0))
        coefidx = np.argsort(coef)
        u = np.sum(tot_acc,0)
       
        best_n = box[np.argmax(u)]
        idx = coefidx[-best_n:]
        
        tr_data = train_data[:,idx]
        te_data = test_data[:,idx]
        print("NN")
        nn_acc, result_nn = dnn(tr_data, train_label, te_data, test_label)
        print("rf")
        rf_acc, result_rf = rfc(tr_data, train_label, te_data, test_label)
        en_acc = 0.0
        for i in range(0,np.shape(te_data)[0]):
            r = np.argmax(result_nn[i]+result_rf[i])
            if r == test_label[i]:
                en_acc += 1
        en_acc /= np.shape(te_data)[0]*0.01
        print("Outer_fold # of features:  %d, Neural network accuracy: %f, Random forests accuracy: %f, Ensemble accuracy: %f" % (best_n, nn_acc, rf_acc, en_acc))
        flag += 1

KF
tot_acc
LSVC


C:\Users\Sophie\Anaconda3\envs\py3.6_bio\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


KeyboardInterrupt: 

In [18]:
        kf = np.random.permutation(int(np.shape(train_data)[0]/Inner_loop)*Inner_loop)
        kf = kf.reshape([Inner_loop]+[-1])
        val_result = np.zeros([np.shape(train_data)[0],48], dtype=np.float32)

In [21]:
        tot_acc = np.zeros([Inner_loop,5], dtype=np.float32)
        lsvc = LinearSVC(C=1, penalty="l1", dual=False).fit(data, np.ravel(label))
        coef = np.squeeze(np.sum(np.square(np.array(lsvc.coef_)), axis=0))
        coefidx = np.argsort(coef)
        

C:\Users\Sophie\Anaconda3\envs\py3.6_bio\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [22]:
coefidx

array([    0, 22072, 22070, ..., 22423, 22427, 22422], dtype=int64)

In [4]:
        for inner_fold in range(0,Inner_loop):
            val_test_ind = kf[inner_fold]
            if inner_fold == Inner_loop-1:
                val_test_ind = np.array(np.concatenate((val_test_ind,np.array(range(int(np.shape(train_data)[0]/Outer_loop)*Outer_loop,np.shape(train_data)[0]),dtype=np.int32)), axis=0),dtype=np.int32)
            
            val_train_ind = np.setdiff1d(np.array(range(0,np.shape(train_data)[0]),dtype=np.int32), val_test_ind)
            val_train = train_data[val_train_ind]
            val_test = train_data[val_test_ind]
            val_train_label = train_label[val_train_ind]
            val_test_label = train_label[val_test_ind]
            temp = 0
            for item in box:
                idx = coefidx[-item:]
                vtrain = val_train[:,idx]
                vtest = val_test[:,idx]
                nn_acc, result_nn = dnn(vtrain, val_train_label, vtest, val_test_label)
                rf_acc, result_rf = rfc(vtrain, val_train_label, vtest, val_test_label)
                en_acc = 0.0
                for i in range(0,np.shape(vtest)[0]):
                    r = np.argmax(result_nn[i]+result_rf[i])
                    if r == val_test_label[i]:
                        en_acc += 1
                en_acc /= np.shape(vtest)[0]*0.01
                tot_acc[inner_fold,temp] = en_acc
                print("Inner_fold # of features: %d, Neural network accuracy: %f, Random forests accuracy: %f, Ensemble accuracy: %f" % (item, nn_acc, rf_acc, en_acc))

                temp += 1
     

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 3319, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-e8ed88c5a5f5>", line 1, in <module>
    for inner_fold in range(0,Inner_loop):
NameError: name 'Inner_loop' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2034, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/ultratb.py", line 1151, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/python3.6/dist-packages/IPyth

NameError: name 'Inner_loop' is not defined

In [30]:
        for inner_fold in range(0,Inner_loop):
            val_test_ind = kf[inner_fold]
            if inner_fold == Inner_loop-1:
                val_test_ind = np.array(np.concatenate((val_test_ind,np.array(range(int(np.shape(train_data)[0]/Outer_loop)*Outer_loop,np.shape(train_data)[0]),dtype=np.int32)), axis=0),dtype=np.int32)
            
            val_train_ind = np.setdiff1d(np.array(range(0,np.shape(train_data)[0]),dtype=np.int32), val_test_ind)
            val_train = train_data[val_train_ind]
            val_test = train_data[val_test_ind]
            val_train_label = train_label[val_train_ind]
            val_test_label = train_label[val_test_ind]
            temp = 0
            for item in box:
                idx = coefidx[-item:]
                vtrain = val_train[:,idx]
                vtest = val_test[:,idx]
                nn_acc, result_nn = dnn(vtrain, val_train_label, vtest, val_test_label)
                rf_acc, result_rf = rfc(vtrain, val_train_label, vtest, val_test_label)
                en_acc = 0.0
                for i in range(0,np.shape(vtest)[0]):
                    r = np.argmax(result_nn[i]+result_rf[i])
                    if r == val_test_label[i]:
                        en_acc += 1
                en_acc /= np.shape(vtest)[0]*0.01
                tot_acc[inner_fold,temp] = en_acc
                print("Inner_fold # of features: %d, Neural network accuracy: %f, Random forests accuracy: %f, Ensemble accuracy: %f" % (item, nn_acc, rf_acc, en_acc))

                temp += 1
     

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

0 epoch Loss: 3.417368
1 epoch Loss: 3.305368
2 epoch Loss: 3.207848
3 epoch Loss: 3.118102
4 epoch Loss: 3.022629
5 epoch Loss: 2.943383
6 epoch Loss: 2.817601
7 epoch Loss: 2.694737
8 epoch Loss: 2.643669
9 epoch Loss: 2.559561
10 epoch Loss: 2.506509
11 epoch Loss: 2.468976
12 epoch Loss: 2.373758
13 epoch Loss: 2.339271
14 epoch Loss: 2.310903
15 epoch Loss: 2.241019
16 epoch Loss: 2.243011
17 epoch Loss: 2.197430
18 epoch Loss: 2.185631
19 epoch Loss: 2.154846
20 epoch Loss: 2.125371
21 epoch Loss: 2.105390
22 epoch Loss: 2.065357
23 epoch Loss: 2.055343
24 epoch Loss: 2.018960
25 epoch Loss: 2.008263
26 epoch

49 epoch Loss: 1.219526
50 epoch Loss: 1.222176
51 epoch Loss: 1.195483
52 epoch Loss: 1.202597
53 epoch Loss: 1.194450
54 epoch Loss: 1.142271
55 epoch Loss: 1.167582
56 epoch Loss: 1.161063
57 epoch Loss: 1.150502
58 epoch Loss: 1.168339
59 epoch Loss: 1.133247
60 epoch Loss: 1.124704
61 epoch Loss: 1.129528
62 epoch Loss: 1.119139
63 epoch Loss: 1.138020
64 epoch Loss: 1.107429
65 epoch Loss: 1.145473
66 epoch Loss: 1.083361
67 epoch Loss: 1.090493
68 epoch Loss: 1.085917
69 epoch Loss: 1.086573
70 epoch Loss: 1.087045
71 epoch Loss: 1.058273
72 epoch Loss: 1.065800
73 epoch Loss: 1.041729
74 epoch Loss: 1.080937
75 epoch Loss: 1.050668
76 epoch Loss: 1.066426
77 epoch Loss: 1.088783
78 epoch Loss: 1.067374
79 epoch Loss: 1.043186
80 epoch Loss: 1.071752
81 epoch Loss: 1.068210
82 epoch Loss: 1.052424
83 epoch Loss: 1.055295
84 epoch Loss: 1.047223
85 epoch Loss: 1.041114
86 epoch Loss: 1.033456
87 epoch Loss: 1.049955
88 epoch Loss: 1.049786
89 epoch Loss: 1.029969
90 epoch Loss: 1

KeyboardInterrupt: 

In [31]:
import scipy.io
mat = scipy.io.loadmat('data.mat')

In [32]:
mat

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Wed Jul 26 12:55:37 2017',
 '__version__': '1.0',
 '__globals__': [],
 'data': array([[0.        , 0.        , 0.        , ..., 0.        , 0.01639344,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.01204819],
        ...,
        [0.        , 0.        , 0.        , ..., 0.0041841 , 0.0125523 ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.09090909, 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.00714286]])}

In [33]:
mat1 = scipy.io.loadmat('label.mat')

In [34]:
mat1

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Thu Jul 06 15:39:02 2017',
 '__version__': '1.0',
 '__globals__': [],
 'label': array([[ 1],
        [ 1],
        [ 1],
        ...,
        [ 7],
        [10],
        [11]], dtype=uint8)}

In [ ]:
t_index = np.random.permutation(int(np.shape(data)[0]/Outer_loop)*Outer_loop)
t_index = np.reshape(t_index, [Outer_loop, -1])  #10*700

# 1. 1*7000
# 2. 10*700 把 t_index reshap row為Outer_loop clown自己安排
#只是把shaphe重塑


# np.shape(data) -- (7002, 35565)
# np.shape(data)[0] -- 7002
# np.random.permutation(5) -- array([2, 1, 3, 4, 0])
# int(np.shape(data)[0]/Outer_loop)*Outer_loop -- 7000

In [ ]:
flag = 0
for test_index in t_index:
    print(test_index)
    if flag == Outer_loop-1:
        
        #同np.array(range(7000,7002)) -- 把7000,7001併入index
        test_index = np.array(np.concatenate((test_index, 
                                              np.array(range(int(np.shape(data)[0]/Outer_loop)*Outer_loop ,
                                                             np.shape(data)[0])))),
                              dtype=np.int32)
        
    #train data照順序丟 --array([   1,    2,    3, ..., 6999, 7000, 7001])
    #先製好亂抽的test data 然後去修改train(把test有的拔掉)
    print("data")
    train_index = np.setdiff1d(np.array(range(0,np.shape(data)[0])), test_index)
    train_data = data[train_index]   #6302
    train_label = label[train_index] #6302
    test_data = data[test_index]     #700
    test_label = label[test_index]   #700
    

        

    tr_data = train_data
    te_data = test_data
    print("rf")
    rf_acc, result_rf = rfc(tr_data, train_label, te_data, test_label)
    en_acc = 0.0
    
    print("Random forests accuracy: %f" % (rf_acc))
    flag += 1